<h1> <center> Exploiting Sentinel-1 imagery time series to detect grasslands in northern Brazil tropical plains</center> </h1>
<h3> <center> Part 3 - Classification </center> </h3>
<center> Arian Ferreira Carneiro </center>
<center>Willian Vieira de Oliveira </center>

## Import required packages

In [1]:
import numpy as np
import pandas as pd
from osgeo import gdal, gdal_array
#from osgeo import osr
from numpy import genfromtxt

# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#import matplotlib.pyplot as plt
#import time

## Input parameters

In [2]:
# Would you like to write the classification maps to files? ['YES', 'NO']
write_files = 'YES'

# Output directory
dir_output = "OUTPUT/Classification_metrics_imgs/"

### Files that contain the metric images and samples

#### Metric images

In [3]:
# Directories to the metric images
dir_NL_MeanImg = "DATA/ee_export/Metric_images/NL_MeanImg.tif"
dir_NL_StdImg = "DATA/ee_export/Metric_images/NL_StdImg.tif"
dir_NL_SumImg = "DATA/ee_export/Metric_images/NL_SumImg.tif"
dir_NL_MinImg = "DATA/ee_export/Metric_images/NL_MinImg.tif"
dir_NL_MaxImg = "DATA/ee_export/Metric_images/NL_MaxImg.tif"
dir_NL_AmpImg = "DATA/ee_export/Metric_images/NL_AmplitudeImg.tif"
dir_NL_CoVarImg = "DATA/ee_export/Metric_images/NL_CoefVariationImg.tif"
dir_NL_FstSlopeImg = "DATA/ee_export/Metric_images/NL_FirstSlopeImg.tif"

dir_Ratio_MeanImg = "DATA/ee_export/Metric_images/Ratio_MeanImg.tif"
dir_Ratio_StdImg = "DATA/ee_export/Metric_images/Ratio_StdImg.tif"
dir_Ratio_SumImg = "DATA/ee_export/Metric_images/Ratio_SumImg.tif"
dir_Ratio_MinImg = "DATA/ee_export/Metric_images/Ratio_MinImg.tif"
dir_Ratio_MaxImg = "DATA/ee_export/Metric_images/Ratio_MaxImg.tif"
dir_Ratio_AmpImg = "DATA/ee_export/Metric_images/Ratio_AmplitudeImg.tif"
dir_Ratio_CoVarImg = "DATA/ee_export/Metric_images/Ratio_CoefVariationImg.tif"
dir_Ratio_FstSlopeImg = "DATA/ee_export/Metric_images/Ratio_FirstSlopeImg.tif"

dir_RGI_MeanImg = "DATA/ee_export/Metric_images/RGI_MeanImg.tif"
dir_RGI_StdImg = "DATA/ee_export/Metric_images/RGI_StdImg.tif"
dir_RGI_SumImg = "DATA/ee_export/Metric_images/RGI_SumImg.tif"
dir_RGI_MinImg = "DATA/ee_export/Metric_images/RGI_MinImg.tif"
dir_RGI_MaxImg = "DATA/ee_export/Metric_images/RGI_MaxImg.tif"
dir_RGI_AmpImg = "DATA/ee_export/Metric_images/RGI_AmplitudeImg.tif"
dir_RGI_CoVarImg = "DATA/ee_export/Metric_images/RGI_CoefVariationImg.tif"
dir_RGI_FstSlopeImg = "DATA/ee_export/Metric_images/RGI_FirstSlopeImg.tif"

dir_VH_MeanImg = "DATA/ee_export/Metric_images/VH_MeanImg.tif"
dir_VH_StdImg = "DATA/ee_export/Metric_images/VH_StdImg.tif"
dir_VH_SumImg = "DATA/ee_export/Metric_images/VH_SumImg.tif"
dir_VH_MinImg = "DATA/ee_export/Metric_images/VH_MinImg.tif"
dir_VH_MaxImg = "DATA/ee_export/Metric_images/VH_MaxImg.tif"
dir_VH_AmpImg = "DATA/ee_export/Metric_images/VH_AmplitudeImg.tif"
dir_VH_CoVarImg = "DATA/ee_export/Metric_images/VH_CoefVariationImg.tif"
dir_VH_FstSlopeImg = "DATA/ee_export/Metric_images/VH_FirstSlopeImg.tif"

dir_VV_MeanImg = "DATA/ee_export/Metric_images/VV_MeanImg.tif"
dir_VV_StdImg = "DATA/ee_export/Metric_images/VV_StdImg.tif"
dir_VV_SumImg = "DATA/ee_export/Metric_images/VV_SumImg.tif"
dir_VV_MinImg = "DATA/ee_export/Metric_images/VV_MinImg.tif"
dir_VV_MaxImg = "DATA/ee_export/Metric_images/VV_MaxImg.tif"
dir_VV_AmpImg = "DATA/ee_export/Metric_images/VV_AmpImg.tif"
dir_VV_CoVarImg = "DATA/ee_export/Metric_images/VV_CoefVariationImg.tif"
dir_VV_FstSlopeImg = "DATA/ee_export/Metric_images/VV_FirstSlopeImg.tif"

# Directories to the sample files
dir_NL_samples = "OUTPUT/NL_AllSamples_metrics.csv"
dir_NL_classes = "OUTPUT/NL_AllSamples_classes.csv"

dir_Ratio_samples = "OUTPUT/Ratio_AllSamples_metrics.csv"
dir_Ratio_classes = "OUTPUT/Ratio_AllSamples_classes.csv"

dir_RGI_samples = "OUTPUT/RGI_AllSamples_metrics.csv"
dir_RGI_classes = "OUTPUT/RGI_AllSamples_classes.csv"

dir_VH_samples = "OUTPUT/VH_AllSamples_metrics.csv"
dir_VH_classes = "OUTPUT/VH_AllSamples_classes.csv"

dir_VV_samples = "OUTPUT/VV_AllSamples_metrics.csv"
dir_VV_classes = "OUTPUT/VV_AllSamples_classes.csv"

# Lists of the directories
filenames = ['NL', 'Ratio', 'RGI', 'VH', 'VV']
filenames_metrics = ['Mean', 'Std', 'Sum', 'Min', 'Max', 'Amp', 'Covar', 'FstSlope']

list_NL_metrics = [dir_NL_MeanImg, dir_NL_StdImg, dir_NL_SumImg, dir_NL_MinImg, dir_NL_MaxImg, dir_NL_AmpImg, dir_NL_CoVarImg, dir_NL_FstSlopeImg]
list_Ratio_metrics = [dir_Ratio_MeanImg, dir_Ratio_StdImg, dir_Ratio_SumImg, dir_Ratio_MinImg, dir_Ratio_MaxImg, dir_Ratio_AmpImg, dir_Ratio_CoVarImg, dir_Ratio_FstSlopeImg]
list_RGI_metrics = [dir_RGI_MeanImg, dir_RGI_StdImg, dir_RGI_SumImg, dir_RGI_MinImg, dir_RGI_MaxImg, dir_RGI_AmpImg, dir_RGI_CoVarImg, dir_RGI_FstSlopeImg]
list_VH_metrics = [dir_VH_MeanImg, dir_VH_StdImg, dir_VH_SumImg, dir_VH_MinImg, dir_VH_MaxImg, dir_VH_AmpImg, dir_VH_CoVarImg, dir_VH_FstSlopeImg]
list_VV_metrics = [dir_VV_MeanImg, dir_VV_StdImg, dir_VV_SumImg, dir_VV_MinImg, dir_VV_MaxImg, dir_VV_AmpImg, dir_VV_CoVarImg, dir_VV_FstSlopeImg]

list_metric_imgs = [list_NL_metrics, list_Ratio_metrics, list_RGI_metrics, list_VH_metrics, list_VV_metrics]

#### Samples

In [4]:
list_samples = [dir_NL_samples, dir_Ratio_samples, dir_RGI_samples, dir_VH_samples, dir_VV_samples]
list_classes = [dir_NL_classes, dir_Ratio_classes, dir_RGI_classes, dir_VH_classes, dir_VV_classes]

## Auxiliary functions

### Function to write each classification map to file

In [5]:
# FUNCTIONS

def openImage(filepath):
    data = gdal.Open(filepath)
    return data


def Write_GeoTiff(file, filename, Nrows, Ncols, geotransform, projection):
    driver = gdal.GetDriverByName('GTiff')
    
    dataset_output = driver.Create(filename, Ncols, Nrows, 1, gdal.GDT_Float32)
    dataset_output.GetRasterBand(1).WriteArray(file)
    
    if geotransform is not None:
        gt = list(geotransform)
        dataset_output.SetGeoTransform(tuple(gt))
    dataset_output.SetProjection(projection)
    
    dataset_output = None

    
def Write_dataframe(df, filename):
    try:
        df.to_csv(filename, sep=',', index=False, encoding='utf-8-sig')
        print("    The dataframe was written to file!")
    except Exception as e:
        print(str(e))

## Data preparation

In this procedure, we convert the metric images to flatten arrays and combine them in an unique dataframe per data type (e.g., NL, VV, VH).

In [6]:
list_metrics = []
#for i in range(1):
for i in range(len(filenames)):
    print("Organizing the ", filenames[i], " data...")
    
    df = pd.DataFrame(columns=filenames_metrics)
    
    print("  Processing metric images...")
    for m in range(len(filenames_metrics)):    
        datacube = openImage(list_metric_imgs[i][m])

        Nrows = datacube.RasterYSize - 6 # We do not consider border pixels. We removed both the first and the last three rows.
        Ncols = datacube.RasterXSize - 6 # We do not consider border pixels. We removed both the first and the last three columns.

        arr = datacube.ReadAsArray(3, 3, Ncols, Nrows) # xoff, yoff, xcount, ycount

        df[filenames_metrics[m]] = arr.flatten()
        
        # Closing image file
        datacube = None
    
    if (write_files == 'YES'):
        filename = dir_output + filenames[i] + '_metrics.csv'
        Write_dataframe(df, filename)
        
        list_metrics.append(filename)
    print("\n")

Organizing the  NL  data...
  Processing metric images...
    The dataframe was written to file!


Organizing the  Ratio  data...
  Processing metric images...
    The dataframe was written to file!


Organizing the  RGI  data...
  Processing metric images...
    The dataframe was written to file!


Organizing the  VH  data...
  Processing metric images...
    The dataframe was written to file!


Organizing the  VV  data...
  Processing metric images...
    The dataframe was written to file!




## Classification: using basic metrics

In this procedure, we perform the classification of the data cube considering the metrics computed for the time series.

In [7]:
#for i in range(1):
for i in range(len(filenames)):
    print("Classifying the ", filenames[i], " data cube...")
    
    pixel_metrics = pd.read_csv(list_metrics[i])
    sample_metrics = pd.read_csv(list_samples[i])
    sample_metrics = sample_metrics.drop(labels='Class', axis=1) # removing the column 'Class' from the dataframe
    sample_classes = pd.read_csv(list_classes[i])
    
    smp_metrics = np.float32(sample_metrics)
    smp_classes = np.float32(sample_classes)
    smp_classes = np.ravel(smp_classes) # converting from column-vector to 1d array (expected by the classifier)
    pixels_metrics = np.float32(pixel_metrics)
    
    # ------------------------- SPLITTING THE SAMPLES INTO TRAINING AND TESTING DATASETS -------------------------
    
    # This scikit-learn function separates the datasets in train and test samples
    smp_metrics_train, smp_metrics_test, smp_classes_train, smp_classes_test = train_test_split(smp_metrics, smp_classes, 
                                                                                                test_size=0.3)
    
    rf = RandomForestClassifier(n_estimators=300, min_samples_leaf = 5, max_features = 5, n_jobs=-1, oob_score=True)
    rf.fit(smp_metrics_train, smp_classes_train) # X and Y parameters (as recognized by the classifier)
    
    Y_train_pred = rf.predict(smp_metrics_train) # Classifies the training samples (resubstitution validation technique)
    acc_train = accuracy_score(smp_classes_train, Y_train_pred) # Computes the accuracy
    print('    Overall accuracy (resubst): ' + str(round(acc_train,4)))
    #print('-------------------')
    
    Y_test_pred = rf.predict(smp_metrics_test) # Classifies the testing samples (hold-out validation technique)
    acc_test = accuracy_score(smp_classes_test, Y_test_pred) # Computes the accuracy
    print('    Overall accuracy (holdout): ' + str(round(acc_test,4)))
    confusion_pred = pd.crosstab(Y_test_pred, smp_classes_test, rownames=['Pred'], colnames=['Actual'], 
                                 margins=False, margins_name="Total") # confusion matrix for the testing dataset
    
    confusion_pred.loc['Accuracies','OA_Resubs'] = acc_train
    confusion_pred.loc['Accuracies','OA_Holdout'] = acc_test

    # -----------------------------------------------------------------------------------------------------------

    class_pred = rf.predict(pixels_metrics) #classification of all pixels
    
    imgPath = "DATA/ee_export/" + filenames[i] + ".tif"
    example_img = gdal.Open(imgPath)

    Nrows = example_img.RasterYSize
    Ncols = example_img.RasterXSize
    GeoTransform = example_img.GetGeoTransform()
    Projection = example_img.GetProjection()

    example_img = None

    classif_array = np.reshape(class_pred, (Nrows-6, Ncols-6))

    # In this study, we disconsidered the border pixels (3-pixel length). Therefore, we need to adjust 
    # the array to follow the same characteristics of the example image (GeoTranform and Projection)
    classif_array_adjusted = np.empty((Nrows, Ncols), np.float32)
    classif_array_adjusted[3:Nrows-3, 3:Ncols-3] = classif_array

    # Writing the classification product and the confusion matrix to files
    if (write_files == 'YES'):
        filename_map = dir_output + filenames[i] + "_Metrics_RFclassification.tif"
        filename_matrix = dir_output + filenames[i] + "_Metrics_RFconfusionMatrix.csv"

        try:
            # Classification map
            Write_GeoTiff(classif_array_adjusted, filename_map, Nrows, Ncols, GeoTransform, Projection)
            
            # Confusion matrix
            confusion_pred.to_csv(filename_matrix, sep=',', index=True, header=True, index_label='Pred/Actual', 
                                  encoding='utf-8-sig')
            print("    The products were written to file!")
            
        except Exception as e:
            print(str(e))
    
    print("\n")

Classifying the  NL  data cube...
    Overall accuracy (resubst): 0.911
    Overall accuracy (holdout): 0.8398
    The products were written to file!


Classifying the  Ratio  data cube...
    Overall accuracy (resubst): 0.8795
    Overall accuracy (holdout): 0.6429
    The products were written to file!


Classifying the  RGI  data cube...
    Overall accuracy (resubst): 0.8687
    Overall accuracy (holdout): 0.6458
    The products were written to file!


Classifying the  VH  data cube...
    Overall accuracy (resubst): 0.913
    Overall accuracy (holdout): 0.832
    The products were written to file!


Classifying the  VV  data cube...
    Overall accuracy (resubst): 0.9052
    Overall accuracy (holdout): 0.8263
    The products were written to file!


